##[KUGGLE] NLP 자연어 처리 과제

**실습 내용**
* 자연어를 효과적으로 분석하기 위해 중요도가 떨어지는 문자 삭제해보기
* 머신러닝으로 텍스트 데이터를 학습하기 위한 수치화 과정(bow) 실습




########## ⭐️ (과제) 문장을 넣어주세요 ############

이 부분에 자연어 처리로 분석해보고 싶은 문장을 넣어 코드를 돌려주세요!


##1. 숫자 및 특수 문자 삭제
* 정규표현식 사용

In [26]:
# 데이터 프레임 생성을 위해 pandas 로드
import pandas as pd

# 특수 문자 및 숫자가 포함된 예시 텍스트 데이터
texts = [ "내 생일은 1월 10일 입니다 ~~@ㅎㅅㅎ@##"
]

# 데이터프레임으로 변환
df = pd.DataFrame(texts, columns=['text'])

In [27]:
# 특수문자 제거
df['text'] = df['text'].replace(r'[^\w\s]', '', regex=True)

In [28]:
# 숫자 제거
df['text'] = df['text'].replace(r'\d+', '', regex=True)

In [29]:
# 결과 출력
print(df)

                text
0  내 생일은 월 일 입니다 ㅎㅅㅎ


##2. 형태소 분석
한국어 형태소 분석은 한국어 텍스트를 가장 작은 의미 단위인 형태소로 분해하는 과정입니다.

형태소의 분해 과정
* 분해: 한국어 문장을 어절 단위로 분해합니다.
* 형태소 분석: 각 어절을 다시 형태소로 세분화 합니다. 이 과정에서 어근, 접미사, 접두사, 조사 등이 구분됩니다,
* 품사 태깅: 분리된 형태소에 품사 정보를 부여합니다. 예를 들어, 명사, 동사, 형용사 등의 품사를 식별합니다.



##KoNLPy란
* KoNLPy는 파이썬 기반 라이브러리로, 한국어 텍스트의 형태소 분석, 품사 태깅 등을 제공합니다.
* KoNLPy에 여러 분석 엔진이 있는데 한국어 텍스트의 경우 대표적으로 okt가 사용됩니다.

##KoNLPy 설치

In [15]:
!pip install --upgrade pip
!pip install JPype1
!pip install konlpy --upgrade

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## 형태소 분석 -실습

In [5]:
from konlpy.tag import Okt

okt = Okt()
# okt.tagset
okt.pos("이 사람은 그 곳에 서 있다.",norm=True, stem=True)

JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.

## 2-1 형태소 분석을 통한 불용어 삭제

**불용어(Stopwords)**: 자연어 처리(NLP)에서 텍스트를 분석할 때 의미를 가지지 않거나 중요도가 낮아서 분석에서 제외되는 단어들을 말합니다.

* ex) 한국어에서의 불용어: "이", "그", "저", "것", "있다", "수", "들", "그리고" 등

* 조사, 접미사, 어미, 구두점 등이 주로 불용어에 해당합니다.

##조사, 어미, 구두점 삭제하는 함수 만들어보기
**KoNLPy 한국어 품사 태깅표:**https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit?gid=0#gid=0

* 위 주소의 태깅표를 참고하여 실습해보시기 바랍니다.
* 아래 작성된 코드는 형태소 분석기를 통해 지정 된 품사를 삭제하는 함수를 만든 것 입니다.
* KoNLPy 품사 태깅표를 확인하여 여러 품사를 삭제하는 함수를 만들 수 있습니다.
***okt는 품사 태깅표의 Twitter Korean Text를 참고하시면 됩니다.**

In [30]:
# 형태소 분석기(Okt) 불러오기

def okt_clean(text):
    clean_text = []
    okt_pos = okt.pos(text, stem=True)
    for txt, pos in okt_pos:
        if pos not in ['JO','Eomi','Punctuation']: # KoNLPy 품사 태깅표 참고하여 조사 어미 구두점 삭제하는 함수 만들어보기
          clean_text.append(txt)
    return " ".join(clean_text)

In [31]:
# 실제 텍스트에 함수 적용해보기
okt_clean(text="자연어 처리는 인공지능의 한 분야입니다.")

NameError: name 'okt' is not defined

##3. 단어 가방(bow: bag of words)

* 자연어 처리에서 텍스트 데이터를 행렬화하여 분석하기 위한 방법입니다.
* 텍스트를 입력하게 되면 텍스트의 모든 단어들을 열로 사용하고 각 문서에서 단어가 얼마나 등장하는지를 행렬로 나타냅니다.
* 단어의 순서나 문법적 구조는 무시하고 단순히 각 단어의 빈도수에 초점을 맞춥니다.

##라이브러리 로드


In [7]:
# 데이터 분석을 위한 pandas, 수치계산을 위한 numpy, 시각화를 위한 seaborn, matplotlib, koreanize_matplotlib 을 로드합니다.
# sklearn.feature_extraction.text의 CountVectorizer 를 통해 BOW 를 생성

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 3-1 CountVectorizer
* CountVectorizer는 사이킷런에서 제공하는bag of words를 만들 수 있는 방법입니다.



In [10]:
corpus = ["코로나 거리두기와 코로나 상생지원금 문의입니다.",
          "지하철 운행시간과 지하철 요금 문의입니다.",
          "지하철 승강장 문의입니다.",
          "코로나 선별진료소 문의입니다.",
          "버스 운행시간 문의입니다.",
          "버스 터미널 위치 안내입니다.",
          "코로나 거리두기 안내입니다.",
          "택시 승강장 문의입니다."
         ]

In [11]:
# CountVectorizer 인스턴스를 만듭니다.
cvect = CountVectorizer()

# 텍스트 데이터를 학습(fit)하고, 이를 BOW 형태로 변환(transform)합니다.
X = cvect.fit_transform(corpus)

# 이제 문장들이 숫자로 변환된 결과를 보기 좋게 표 형태로 만듭니다.
#    - X.toarray()는 각 문장을 숫자로 변환한 결과를 보기 좋게 바꿔줍니다.
#    - columns=cvect.get_feature_names_out()는 각 열(컬럼)에 해당하는 단어의 이름을 표에 추가해줍니다.
pd.DataFrame(X.toarray(), columns=cvect.get_feature_names_out())

,거리두기,거리두기와,문의입니다,버스,상생지원금,선별진료소,승강장,안내입니다,요금,운행시간,운행시간과,위치,지하철,코로나,택시,터미널
0,0,1,1,0,1,0,0,0,0,0,0,0,0,2,0,0
1,0,0,1,0,0,0,0,0,1,0,1,0,2,0,0,0
2,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0
3,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
4,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0
5,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1
6,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
7,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0


##CountVectorizer 실습

In [12]:
corpus = ["코로나 거리두기와 코로나 상생지원금 문의입니다.",
          "지하철 운행시간과 지하철 요금 문의입니다.",
          "지하철 승강장 문의입니다.",
          "코로나 선별진료소 문의입니다.",
          "버스 운행시간 문의입니다.",
          "버스 터미널 위치 안내입니다.",
          "코로나 거리두기 안내입니다.",
          "택시 승강장 문의입니다."
         ]

In [13]:
# sklearn.feature_extraction.text의 CountVectorizer 를 통해 BOW 를 생성
from sklearn.feature_extraction.text import CountVectorizer


cvect = CountVectorizer()
X = cvect.fit_transform(corpus)
pd.DataFrame(X.toarray(), columns=cvect.get_feature_names_out())

,거리두기,거리두기와,문의입니다,버스,상생지원금,선별진료소,승강장,안내입니다,요금,운행시간,운행시간과,위치,지하철,코로나,택시,터미널
0,0,1,1,0,1,0,0,0,0,0,0,0,0,2,0,0
1,0,0,1,0,0,0,0,0,1,0,1,0,2,0,0,0
2,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0
3,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
4,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0
5,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1
6,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
7,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0


## 3-2 N-grams
* bow에서 토큰을 몇 개 사용할 것인지를 구분합니다. 지정한 n개의 숫자 만큼의 토큰을 묶어서 사용합니다.
* 예를 들어 (1,1)이라면 1개의 토큰을 (2,3)이라면 2~3개의 토큰을 사용합니다.

* 토큰: 자연어 처리에서 텍스트를 분석하기 위해 가장 작은 단위로 나눈 것 입니다.

* 토큰을 (1,2) 설정하게 되면 기존의 하나의 단어로 행렬의 열을 구성하였지만, 하나의 단어 뿐만 아니라 2개의 단어(연속된 단어만 가능)로 이루어진 열 또한 만들어서 행렬을 생성합니다.

In [21]:
#N-grams를 적용하여 bow실습
cvect = CountVectorizer(ngram_range=(1,2))
X = cvect.fit_transform(corpus)
dtm = X.toarray()
pd.DataFrame(dtm, columns=cvect.get_feature_names_out()).style.background_gradient()

,거리두기,거리두기 안내입니다,거리두기와,거리두기와 코로나,문의입니다,버스,버스 운행시간,버스 터미널,상생지원금,상생지원금 문의입니다,선별진료소,선별진료소 문의입니다,승강장,승강장 문의입니다,안내입니다,요금,요금 문의입니다,운행시간,운행시간 문의입니다,운행시간과,운행시간과 지하철,위치,위치 안내입니다,지하철,지하철 승강장,지하철 요금,지하철 운행시간과,코로나,코로나 거리두기,코로나 거리두기와,코로나 상생지원금,코로나 선별진료소,택시,택시 승강장,터미널,터미널 위치
0,0,0,1,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,1,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1
6,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
7,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


## 3-3 max_features
* 기본값 = None
* CountVectoriezr가 학습할 기능(어휘)의 양 제한
* corpus중 빈도수가 가장 높은 순으로 해당 개수만큼만 추출
* max_features = n 입력시 bow 행렬의 컬럼이 최대 n개까지만 생성됩니다.

In [20]:
# 10개의 단어로만 컬럼 형성하기
cvect = CountVectorizer(ngram_range=(1, 2),max_features= 10 )
X = cvect.fit_transform(corpus)
dtm = X.toarray()

# df_dtm.sum 으로 빈도수 합계 구하기
pd.DataFrame(dtm, columns=cvect.get_feature_names_out())



,문의입니다,버스,승강장,승강장 문의입니다,안내입니다,위치 안내입니다,지하철,지하철 승강장,지하철 요금,코로나
0,1,0,0,0,0,0,0,0,0,2
1,1,0,0,0,0,0,2,0,1,0
2,1,0,1,1,0,0,1,1,0,0
3,1,0,0,0,0,0,0,0,0,1
4,1,1,0,0,0,0,0,0,0,0
5,0,1,0,0,1,1,0,0,0,0
6,0,0,0,0,1,0,0,0,0,1
7,1,0,1,1,0,0,0,0,0,0
